# Histogram of EM Data with Automatic Min/Max Visualization 

This notebook is for exploration of automatic computation of Min/Max parameters for adjust with image intestity range used to visualized electron microscopy image data.

In [1]:
input_filename = "/Users/blowekamp/Desktop/20130412-Vn-poxtomo_8/20130412-Vn-poxtomo_8_full_rec.mrc"

In [2]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
%matplotlib notebook

In [3]:
img = sitk.ReadImage(input_filename)

In [ ]:
stats = sitk.StatisticsImageFilter()
stats.Execute(img)
print(stats)

In [ ]:
a_img = sitk.GetArrayViewFromImage(img).flatten()
bins = 2**np.iinfo(a_img.dtype).bits
h, b = np.histogram(a_img, bins=bins, range=(np.iinfo(a_img.dtype).min, np.iinfo(a_img.dtype).max + 1))

In [ ]:
def rescale_min_max(img, vmin, vmax):
    ss_filter = sitk.ShiftScaleImageFilter()
    ss_filter.SetShift( -float(vmin) )
    ss_filter.SetScale( 255.0/(vmax-vmin) )
    ss_filter.SetOutputPixelType(sitk.sitkUInt8)
    return ss_filter.Execute(img)

In [ ]:

def plot_range_options(img, range_mult=3, percentile_crop=10, bins=1024, fig_filename=None):


    fa_img = sitk.GetArrayViewFromImage(img).flatten()
    median = np.median(fa_img)
    mad = np.median(np.abs(fa_img-median))
    mean = np.mean(fa_img)
    sigma = np.sqrt(np.var(fa_img))


    mad_range =  (median-range_mult*mad, median+range_mult*mad)
    percentile_range = (np.percentile(a_img,percentile_crop*.5), np.percentile(a_img,100-percentile_crop*.5))

    
    sigma_range = (mean-range_mult*sigma, 
                   mean+range_mult*sigma)
    
    min_max_range = (np.min(fa_img), np.max(fa_img))

    fig = plt.figure(figsize=(8, 8), dpi=240)
    ax = fig.add_subplot(2, 4, (1, 4))

    x = np.linspace(min(fa_img), max(fa_img), bins)
    plt.plot(x, norm.pdf(x, mean, sigma), color='k', linestyle='--', alpha=0.5, label=f'Fit Gaussian')

    plt.hist( fa_img, bins=bins, density=True, color='b')

    plt.axvline(x=mad_range[0], color='r', alpha=0.5, label=f'{range_mult} Median Absolute Deviation')
    plt.axvline(x=mad_range[1], color='r', alpha=0.5)
    plt.axvline(x=median, color='r', alpha=0.5, linestyle='--', label=f'Median')
    print(f"Median: {median} {range_mult} Median Absolute Deviation: {mad_range}")


    plt.axvline(x=percentile_range[0], color='g', alpha=0.5, label=f'Middle {100-percentile_crop} Percentile')
    plt.axvline(x=percentile_range[1], color='g', alpha=0.5)
    print(f"Middle {100-percentile_crop} Percentile: {percentile_range}" )

    plt.axvline(x=sigma_range[0], color='y', alpha=0.5, label=f'{range_mult} Sigma')
    plt.axvline(x=sigma_range[1], color='y', alpha=0.5)
    print(f"{range_mult} Sigma: {sigma_range}" )
    
    plt.axvline(x=min_max_range[0], color='k', alpha=0.5, label=f'Min/Max')
    plt.axvline(x=min_max_range[1], color='k', alpha=0.5)
    print(f"Min/Max: {min_max_range}" )


    
   

    plt.legend(fontsize=5)
    plt.title("Data Ranges and Histogram")

    
    img_slice = img[:,img.GetSize()[1]//2,:]
    
    ax = fig.add_subplot(2, 4, 5)
    plt.imshow(sitk.GetArrayViewFromImage(rescale_min_max(img_slice,*mad_range)), cmap='gray')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.title(f'{range_mult} Median Absolute Deviation', color='r', fontsize=5)
    
    ax = fig.add_subplot(2, 4, 6)
    plt.imshow(sitk.GetArrayViewFromImage(rescale_min_max(img_slice,*percentile_range)), cmap='gray')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.title(f'Middle {100-percentile_crop} Percentile', color='g', fontsize=5)
    
    ax = fig.add_subplot(2, 4, 7)
    plt.imshow(sitk.GetArrayViewFromImage(rescale_min_max(img_slice,*sigma_range)), cmap='gray')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.title(f'{range_mult} Sigma', color='y', fontsize=5)
    
    
    ax = fig.add_subplot(2, 4, 8)
    plt.imshow(sitk.GetArrayViewFromImage(rescale_min_max(img_slice, *min_max_range)), cmap='gray')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.title(f'Data Min/Max', color='k', fontsize=5)
    
    plt.show()
    if fig_filename:
        plt.savefig(fig_filename)

In [ ]:
plot_range_options(img, range_mult=3, percentile_crop=10, bins=1024, fig_filename="figure.pdf")

In [4]:
a_img = sitk.GetArrayViewFromImage(img).flatten()
bins = 2**np.iinfo(a_img.dtype).bits

h, b = np.histogram(list(), bins=bins, range=(np.iinfo(a_img.dtype).min, np.iinfo(a_img.dtype).max + 1))
       

In [ ]:
def stream_build_histogram(filename:str, histogram_bin_edges, extract_axis = 1, density = False):
    reader = sitk.ImageFileReader()
    reader.SetFileName(filename)
    reader.ReadImageInformation()
      
    extract_index = [0]*reader.GetDimension()

    extract_size = list(reader.GetSize())
    extract_size[extract_axis] = 0
    reader.SetExtractSize(extract_size)

    h = np.zeros(len(histogram_bin_edges)-1, dtype=np.int64)
    
    for i in range(reader.GetSize()[extract_axis]):
        extract_index[extract_axis] = i
        reader.SetExtractIndex(extract_index)
        img = reader.Execute()
        
        # accumulate histogram density/weights
        h += np.histogram( sitk.GetArrayViewFromImage(img).flatten(), bins=histogram_bin_edges, density=density)[0]
    
    return h, np.array(np.histogram_bin_edges)

In [ ]:
bin_edges=range(np.iinfo(a_img.dtype).min, np.iinfo(a_img.dtype).max + 2)
h, b = stream_build_histogram(input_filename, bin_edges)

In [ ]:
len(b2)

In [ ]:
len(bin_edges)